# Phytoplankton Primary Composition end-to-end: HPO → train → evaluate → SHAP (dataset-agnostic)

This notebook runs the full pipeline using the installed `phyx` package.

**Prereqs**
- Environment created/activated
- Package installed: `pip install -e . --no-deps`
- Dataset folder with Parquet files:
    * df_rrs.pqt # feature columns (e.g., Rrs_* and/or ancillary features)
    * df_phyto.pqt # targets: dia, chl, cya, coc, din, pha, tot_chla
    * df_env.pqt # optional (aligned rows)


**Flow**
1. Load features/targets (and optional env)
2. Train/test split
3. Optuna HPO (KFold inside objective)
4. Train on full training split with best params
5. Evaluate on test split; save metrics
6. SHAP values + per-target summary plots
7. (Optional) Train final production model on all data


---
## 1) Setup: imports, logging, paths, and versions

In [1]:
from pathlib import Path
import json

from loguru import logger

# Configure concise logging for notebooks
logger.remove()
logger.add(lambda msg: print(msg, end=""), level="INFO")

# --- Set your paths & options here ---
#DATA_DIR = Path("your_data_path")       # <-- change to your dataset folder
DATA_DIR = Path.home() / 'data/craig_pfc_2023/step_2_cleaned'
assert DATA_DIR.exists(), f"Data directory {DATA_DIR} does not exist!"
OUTDIR   = Path("artifacts/pace_run")     # where outputs are written
N_TRIALS = 30                             # use 5–10 for a quick smoke test
TEST_SIZE = 0.20
SEED = 42

OUTDIR.mkdir(parents=True, exist_ok=True)

# Versions (optional but helpful)
import numpy, pandas, xgboost, sklearn, shap, optuna
logger.info(
    "Versions → numpy={}, pandas={}, xgboost={}, sklearn={}, shap={}, optuna={}\n",
    numpy.__version__, pandas.__version__, xgboost.__version__,
    sklearn.__version__, shap.__version__, optuna.__version__
)


/Users/erdemkarakoylu/miniconda3/envs/phyx/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-09-02 21:41:09.075 | INFO     | __main__:<module>:23 - Versions → numpy=2.2.6, pandas=2.2.3, xgboost=2.1.4, sklearn=1.6.1, shap=0.46.0, optuna=4.5.0



## 2) Load data

Loads `df_rrs.pqt` (features) and `df_phyto.pqt` (targets). If `df_env.pqt` exists, it’s concatenated to features. Basic sanity checks ensure numeric columns and aligned rows.

In [3]:
from phyx.pipeline.data_loader import DataLoader
import pandas as pd

dl = DataLoader(
    DATA_DIR, rrs_file='df_rrs_pace_sub.pqt', env_file='df_env.pqt',
    phy_file='df_phy.pqt')
X_df, Xenv_df, Y_df = dl.load_data()
if Xenv_df is not None:
    X_df = X_df.join(Xenv_df)

# Sanity checks
non_numeric = [c for c in X_df.columns if not pd.api.types.is_numeric_dtype(X_df[c])]
if non_numeric:
    raise TypeError(f"Non-numeric feature columns detected (first few): {non_numeric[:5]}")
assert X_df.shape[0] == Y_df.shape[0], "Row count mismatch between X and Y."

logger.info("Loaded X: {} rows × {} cols | Y: {} rows × {} targets\n",
            X_df.shape[0], X_df.shape[1], Y_df.shape[0], Y_df.shape[1])

2025-09-02 21:46:41.145 | INFO     | phyx.pipeline.data_loader:load_data:27 - Loaded X=%s, Y=%s%s
2025-09-02 21:46:41.231 | INFO     | __main__:<module>:17 - Loaded X: 1261607 rows × 44 cols | Y: 1261607 rows × 7 targets



## 3) Train/test split + schema guard

Create a deterministic split and verify that feature **names and order** match between train and test.

In [4]:
from sklearn.model_selection import train_test_split

Xtr, Xte, Ytr, Yte = train_test_split(X_df, Y_df, test_size=TEST_SIZE, random_state=SEED)
logger.info("Split → train: {} rows, test: {} rows\n", Xtr.shape[0], Xte.shape[0])

# Fail fast if schema differs
if list(Xtr.columns) != list(Xte.columns):
    missing = [c for c in Xtr.columns if c not in Xte.columns]
    extra   = [c for c in Xte.columns if c not in Xtr.columns]
    moved   = [c for c in Xtr.columns if c in Xte.columns and Xtr.columns.get_loc(c) != Xte.columns.get_loc(c)]
    raise ValueError(f"Feature schema mismatch.\nMissing in test: {missing}\nExtra in test: {extra}\nReordered: {moved}")

2025-09-02 21:53:54.008 | INFO     | __main__:<module>:4 - Split → train: 1009285 rows, test: 252322 rows



## 4) Hyperparameter optimization (Optuna)

Runs KFold CV inside the objective for each trial. Saves `best_params.json`.


In [ ]:
import optuna
from phyx.pipeline.optuna_hpo import objective

study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, Xtr, Ytr), n_trials=N_TRIALS, show_progress_bar=False)

best_params = {**study.best_trial.params, "objective": "reg:squarederror"}
best_path = OUTDIR / "best_params.json"
best_path.write_text(json.dumps(best_params, indent=2))

logger.info("Best RMSE={:.4f}\n", study.best_value)
logger.info("Saved best params → {}\n", best_path)
best_params


## 5) Train on full training split with best params; save model

In [ ]:
from phyx.pipeline.model_trainer import XGBoostTrainer

trainer = XGBoostTrainer(params=best_params)
trainer.fit_full(Xtr, Ytr)                    # remembers feature schema if DataFrames were used
MODEL_PATH = OUTDIR / "model.pkl"
trainer.save(MODEL_PATH)

## 6) Evaluate on test split; save metrics

Computes MSE, RMSE, MAE, and R² on the held-out test set and writes `metrics.json`.


In [ ]:
from phyx.pipeline.model_evaluator import ModelEvaluator

evaluator = ModelEvaluator()
yhat = trainer.predict(Xte)   # will validate schema if DataFrame
mse, r2, mae, rmse = evaluator.evaluate(Yte.to_numpy(), yhat)

metrics = {"mse": float(mse), "rmse": float(rmse), "mae": float(mae), "r2": float(r2)}
(OUTDIR / "metrics.json").write_text(json.dumps(metrics, indent=2))
logger.info("Metrics → {}\n", metrics)
metrics


## 7) SHAP on test split: values + per-target summary plots

Computes SHAP values on a sample of the test set and saves summary plots (`shap_summary_target_*.png`). Displays the first few inline.

In [ ]:
from phyx.explain.shap_runner import run_shap_and_plots
from IPython.display import Image, display
import glob

npz_path = run_shap_and_plots(MODEL_PATH, Xte, list(Y_df.columns), OUTDIR, nsamples=2000)
plot_paths = sorted(glob.glob(str(OUTDIR / "shap_summary_target_*.png")))
logger.info("Saved {} SHAP summary plots → {}\n", len(plot_paths), OUTDIR)

# Preview up to 3 plots
for p in plot_paths[:3]:
    display(Image(filename=p))


## 8) (Optional) Train a final “production” model on **all data**

This refits using the tuned hyperparameters on `(train + test)`. It **does not** overwrite the evaluated model; it saves `model_production.pkl`. DO NOT report metrics for this model (test was used for training) unless new data becomes available.

In [ ]:
FIT_ALL = True  # set False to skip

if FIT_ALL:
    trainer.fit_full(X_df, Y_df)
    PROD_MODEL = OUTDIR / "model_production.pkl"
    trainer.save(PROD_MODEL)
    logger.info("Saved production model trained on ALL data → {}\n", PROD_MODEL)
else:
    logger.info("Skipped production fit (FIT_ALL=False)\n")